Aqui estão os trem de estudo

In [2]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [3]:
df_ori_dest = pd.read_csv('./dados/origem_porto.csv')
df_ori_trans = pd.read_csv('./dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('./dados/transbordo_porto.csv')

In [4]:
df_supply = pd.read_csv('./dados/supply.csv')
df_cap_port = pd.read_csv('./dados/cap_porto.csv')
df_cap_trans = pd.read_csv('./dados/cap_transbordo.csv')
df_demand = pd.read_csv('./dados/demand.csv')

In [5]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [6]:
df_supply

,0
0,479
1,594
2,915
3,677


In [8]:
df_demand

,0
0,676
1,803


In [9]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

ways = {}
for i in M:
    ways[i] = {k for k in K}

cost = {}
for i in N:
    for j in M:
        cost[i, j, 0] = df_ori_dest[str(j - M[0])][i]
for i in N:
    for k in K:
        for j in M:
            cost[i, j, k] = df_ori_trans[str(k - K[0])][i] + df_trans_porto[str(j - M[0])][k - K[0]]

K.append(0)

In [10]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]
print(oferta_total, demanda_total)

2665 1479


In [11]:
m = gp.Model("probrema")

In [12]:
X = {}
for i in N:
    for j in M:
        for k in K:
            if not k in ways[j]: continue
            for o in O:
                X[i, j, k, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}_{}".format(i, j, k, o))

# for k in K:
#     for j in M:
#         for o in O:
#             X[k, j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}".format(k, j, o))

# for i in N:
#     for j in M:
#         for o in O:
#             X[i, j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}".format(i, j, o))

In [13]:
# m.setObjective(
#     gp.quicksum(X[i, k, o] * cost[i, k] for i in N for k in K for o in O) + \
#     gp.quicksum(X[k, j, o] * cost[k, j] for k in K for j in M for o in O) + \
#     gp.quicksum(X[i, j, o] * cost[i, j] for i in N for j in M for o in O)
#     ,
#     sense=gp.GRB.MINIMIZE)

m.setObjective(
    gp.quicksum((X[i, j, k, o] * cost[i, j, k]) for i in N for j in M for k in K for o in O if k in ways[j]),
    sense=gp.GRB.MINIMIZE
)

In [14]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) <= supply[i]
        )

In [17]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) == demand[o]
        )

In [20]:
m.addConstrs(
    gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) >= 0 for o in O
)

# m.addConstrs(
#     gp.quicksum(X[i, k, o] for i in N for k in K) >= 0 for o in O
# )

# m.addConstrs(
#     gp.quicksum(X[k, j, o] for k in K for j in M) >= 0 for o in O
# )

{9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [21]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 48 columns and 144 nonzeros
Model fingerprint: 0x9161dc73
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 9e+02]
Presolve removed 2 rows and 40 columns
Presolve time: 0.01s
Presolved: 6 rows, 8 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9468300e+04   1.159153e+02   0.000000e+00      0s
       2    3.9877208e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.987720808e+04


In [67]:
m.write("out.sol")

GurobiError: Unable to retrieve attribute 'X'